In [ ]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.graph_objects as go

import sklearn.metrics
import scipy.integrate as integrate
import scipy.optimize as sco

import fluids as fds
import ht 

import general as gen
import model as ty
import proc as pr
import heat_transfer as bht
import plot_functions as pfun
# importing sys
import sys
sys.path.insert(0,r'C:\Users\valen\Dropbox (DualSun)\PC\Documents\GitHub\RD-systems-and-test-benches')
import SK_standard as sk

import networkx as nx

In [ ]:
input_name = r'\Inputs_V4.41-NI-glass-backsheet.xlsx'

par = pr.import_input(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Inputs',input_name)
hyp = pr.create_dict_from_excel(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Hypotheses','\Model_hypotheses.xlsx','Main')
data = r'\230104_TUV_test_conditions_V4.41_dark_nowind.xlsx'
condi = pr.create_inputs_from_excel(data,par,hyp)

In [ ]:
df_res,X,list_res,list_df,list_df_historic = ty.simu_condi(par,hyp,condi)

In [ ]:
# u_list = [0.7,2.1,3.7]
u_list = [0.3]

color_list = ['blue','red','green','orange','cyan','purple']

popt_mod_list,pcov_mod_list,popt_tuv_list,pcov_tuv_list = pr.linear_interpolation(u_list,df_res,condi)

pfun.plot_model_tuv(u_list,df_res,condi,popt_mod_list,popt_tuv_list,color_list)

In [ ]:
pr.disp_html(df_res)

In [ ]:
pr.display_a_i(X)

In [ ]:
[tab,X] = sk.find_SK_SST(condi,par['A_G'],columns_list=[0,1,0,0,0,0,0,0,0])
[tab_m,X_m] = sk.find_SK_SST(df_res,par['A_G'],columns_list=[0,1,0,0,0,0,0,0,0])

In [ ]:
X_df = pd.DataFrame(X[0],index = ['eta0,hem','a1','a2','a3','a4','a5','a6','a7','a8'],columns=['TÜV'])

In [ ]:
X_df['Model'] = X_m[0]

In [ ]:
X_df = pd.concat(X_df,)

In [ ]:
X_df = X_df.T

In [ ]:
X_df

In [ ]:
X_df[0].name = 'Model'

In [ ]:
X_df

In [ ]:
mse = sklearn.metrics.mean_squared_error(df_res['Q_dot / A_G'],condi['Q_dot / A_G'])
print(math.sqrt(mse))